In [112]:
import csv
import json
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string
import pickle
import collections
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from collections import Counter

lmtz = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import os
os.getcwd()

'C:\\Users\\boom-\\Documents\\GitHub\\stat628-module3-group4-fall2022-uwmadison\\code\\Review Analysis'

### 1. Import review_with_rating from EDA part

In [3]:
bars_review = pd.read_csv("../../data/bars_list_ca.csv").drop(columns=['Unnamed: 0'])
bars_review = pd.DataFrame(data = bars_review, columns = ["business_id","stars_x","text"])
bars_review = bars_review.sort_values(by = ["business_id"]).reset_index().drop(columns = ["index"])

In [4]:
bars_review

,business_id,stars_x,text
0,-BdYhP-12elmFV7oB1iv4A,5.0,"Great beer selection, flat breads are great, g..."
1,-BdYhP-12elmFV7oB1iv4A,5.0,Great new addition to the Mesa. Great selecti...
2,-BdYhP-12elmFV7oB1iv4A,5.0,Corner Tap has been nothing but the friendlies...
3,-BdYhP-12elmFV7oB1iv4A,5.0,Awesome spot and much needed on the Mesa! Lov...
4,-BdYhP-12elmFV7oB1iv4A,5.0,This is the place to go and decompress. The pe...
...,...,...,...
44688,zkMqKD6IZPcU10g0a2APmg,5.0,First time to Santa Barbara and this shop but ...
44689,zkMqKD6IZPcU10g0a2APmg,5.0,La Aroma De Havana is a great cigar lounge I w...
44690,zkMqKD6IZPcU10g0a2APmg,5.0,Cigar and beer lover? Then it's a must stop wh...
44691,zkMqKD6IZPcU10g0a2APmg,5.0,I would vacation in Santa Barbara just to smok...


In [5]:
review_with_rating = pd.read_csv("../../data/review_with_rating.csv")

In [6]:
review_with_rating

,rating,review_words
0,5.0,"['bit', 'weary', 'trying', 'shellfish', 'compa..."
1,5.0,"['great', 'beautiful', 'spot', 'wine', 'tastin..."
2,3.0,"['tough', 'one', 'merit', 'wine', 'fairly', 'a..."
3,5.0,"['love', 'trying', 'fresh', 'seafood', 'pier',..."
4,5.0,"['super', 'delish', 'frill', 'great', 'sea', '..."
...,...,...
44688,5.0,"['favorite', 'brunch', 'spot', 'santa', 'barba..."
44689,5.0,"['combo', 'meat', 'plate', 'good', 'ton', 'foo..."
44690,2.0,"['table', 'got', 'many', 'weird', 'look', 'emp..."
44691,5.0,"['could', 'give', '5', 'star', 'would', 'reser..."


### 2. Get the adjectives from review_words

In [7]:
nltk.pos_tag(["tough","got","star"])

[('tough', 'JJ'), ('got', 'VBD'), ('star', 'NN')]

In [8]:
adjectives = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    adj = [word for word, tag in tag if tag == "JJ"]
    adjectives.append(adj)

In [9]:
adjectives

[['weary',
  'shellfish',
  'many',
  'monterey',
  'feast',
  'captive',
  'high',
  'perennial',
  'good',
  'initial',
  'small',
  'plan',
  'peak',
  'special',
  'local',
  'awesome',
  'great',
  'hearty',
  'good',
  'much',
  'shrimp',
  'bisque',
  'prattling',
  'glorious',
  'white',
  'thick',
  'separate',
  'good',
  'great',
  'satisfied',
  'special',
  'overall',
  'great',
  'favorite',
  'safe'],
 ['great', 'beautiful', 'wine', 'tried', 'great', 'good'],
 ['tough',
  'average',
  'understand',
  'lable',
  'available',
  'wonderful',
  'nicer',
  'enjoy',
  'super',
  'expensive',
  'free',
  'wine',
  'winery',
  'give',
  'complimentary',
  'carr',
  'new',
  'wine',
  'limited',
  'beautiful',
  'avoid',
  'much',
  'wine',
  'urban'],
 ['fresh',
  'wharf',
  'fresh',
  'gorgeous',
  'top',
  'beautiful',
  'wrong',
  'good',
  'lunchearly',
  'table',
  'outdoor',
  'right',
  'sunscreen',
  'local',
  'salad',
  'steal',
  'huge',
  'delicious',
  'fresh',
  's

### 3. Create dictionary counting adjectives from reviews

In [10]:
# Collect all adjective
all_adj = [adj for i in adjectives for adj in i]
adj_freq = dict(collections.Counter(all_adj))

In [11]:
adj_freq

{'weary': 18,
 'shellfish': 403,
 'many': 2894,
 'monterey': 6,
 'feast': 10,
 'captive': 3,
 'high': 1662,
 'perennial': 3,
 'good': 23313,
 'initial': 86,
 'small': 3718,
 'plan': 26,
 'peak': 45,
 'special': 3438,
 'local': 3279,
 'awesome': 2563,
 'great': 25744,
 'hearty': 31,
 'much': 3311,
 'shrimp': 774,
 'bisque': 26,
 'prattling': 1,
 'glorious': 30,
 'white': 1475,
 'thick': 265,
 'separate': 176,
 'satisfied': 178,
 'overall': 2519,
 'favorite': 3598,
 'safe': 242,
 'beautiful': 2286,
 'wine': 3433,
 'tried': 604,
 'tough': 325,
 'average': 571,
 'understand': 152,
 'lable': 1,
 'available': 826,
 'wonderful': 2756,
 'nicer': 52,
 'enjoy': 396,
 'super': 1966,
 'expensive': 933,
 'free': 1386,
 'winery': 95,
 'give': 1126,
 'complimentary': 442,
 'carr': 23,
 'new': 3196,
 'limited': 234,
 'avoid': 106,
 'urban': 111,
 'fresh': 6068,
 'wharf': 122,
 'gorgeous': 415,
 'top': 2239,
 'wrong': 1125,
 'lunchearly': 5,
 'table': 5896,
 'outdoor': 1084,
 'right': 2246,
 'sunscreen

In [12]:
len(adj_freq)

18020

### 4. Create matrix of frequency

In [13]:
corpus = pd.DataFrame(data = list(bars_review["text"]), columns = ["text"])
vectorizer = CountVectorizer(vocabulary = list(adj_freq.keys()), min_df = 0)
freq_matrix = pd.DataFrame(data = vectorizer.fit_transform(corpus["text"].values).toarray(), columns = vectorizer.get_feature_names())

In [14]:
corpus

,text
0,"Great beer selection, flat breads are great, g..."
1,Great new addition to the Mesa. Great selecti...
2,Corner Tap has been nothing but the friendlies...
3,Awesome spot and much needed on the Mesa! Lov...
4,This is the place to go and decompress. The pe...
...,...
44688,First time to Santa Barbara and this shop but ...
44689,La Aroma De Havana is a great cigar lounge I w...
44690,Cigar and beer lover? Then it's a must stop wh...
44691,I would vacation in Santa Barbara just to smok...


In [15]:
freq_matrix

,weary,shellfish,many,monterey,feast,captive,high,perennial,good,initial,...,not_fear,focusing,cassava,privileged,tupperware,biodegradable,trendyromantic,delicious30,agh,hasslefree
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
44689,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Fit the model using Naive Bayes classifier

In [16]:
# First we classify stars to positive/negative (4 stars or more are positive)
bars_review["stars_x"] = np.where(bars_review["stars_x"] >= 4, "positive", "negative")

In [17]:
bars_review

,business_id,stars_x,text
0,-BdYhP-12elmFV7oB1iv4A,positive,"Great beer selection, flat breads are great, g..."
1,-BdYhP-12elmFV7oB1iv4A,positive,Great new addition to the Mesa. Great selecti...
2,-BdYhP-12elmFV7oB1iv4A,positive,Corner Tap has been nothing but the friendlies...
3,-BdYhP-12elmFV7oB1iv4A,positive,Awesome spot and much needed on the Mesa! Lov...
4,-BdYhP-12elmFV7oB1iv4A,positive,This is the place to go and decompress. The pe...
...,...,...,...
44688,zkMqKD6IZPcU10g0a2APmg,positive,First time to Santa Barbara and this shop but ...
44689,zkMqKD6IZPcU10g0a2APmg,positive,La Aroma De Havana is a great cigar lounge I w...
44690,zkMqKD6IZPcU10g0a2APmg,positive,Cigar and beer lover? Then it's a must stop wh...
44691,zkMqKD6IZPcU10g0a2APmg,positive,I would vacation in Santa Barbara just to smok...


In [18]:
model = MultinomialNB()
model.fit(freq_matrix, bars_review['stars_x'])

MultinomialNB()

In [19]:
# Test the prediction performance

def prediction(adj):
    return model.predict(vectorizer.transform([adj]))[0]

In [20]:
prediction("worst")

'negative'

In [21]:
adj_classifier = {}
for adj in adj_freq.keys():
    adj_classifier[adj] = prediction(adj)

In [22]:
adj_classifier

{'weary': 'positive',
 'shellfish': 'positive',
 'many': 'positive',
 'monterey': 'positive',
 'feast': 'positive',
 'captive': 'negative',
 'high': 'positive',
 'perennial': 'positive',
 'good': 'positive',
 'initial': 'positive',
 'small': 'positive',
 'plan': 'positive',
 'peak': 'positive',
 'special': 'positive',
 'local': 'positive',
 'awesome': 'positive',
 'great': 'positive',
 'hearty': 'positive',
 'much': 'positive',
 'shrimp': 'positive',
 'bisque': 'positive',
 'prattling': 'positive',
 'glorious': 'positive',
 'white': 'positive',
 'thick': 'positive',
 'separate': 'positive',
 'satisfied': 'positive',
 'overall': 'positive',
 'favorite': 'positive',
 'safe': 'positive',
 'beautiful': 'positive',
 'wine': 'positive',
 'tried': 'positive',
 'tough': 'positive',
 'average': 'negative',
 'understand': 'negative',
 'lable': 'negative',
 'available': 'positive',
 'wonderful': 'positive',
 'nicer': 'positive',
 'enjoy': 'positive',
 'super': 'positive',
 'expensive': 'positive'

### 6. Get the nouns from review_words and count

In [23]:
nouns = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    noun = [word for word, tag in tag if tag == "NN"]
    nouns.append(noun)

In [24]:
# Collect all nouns
all_noun = [noun for i in nouns for noun in i]
noun_freq = dict(collections.Counter(all_noun))

In [25]:
noun_freq

{'bit': 3072,
 'company': 510,
 'see': 732,
 'cannery': 2,
 'audience': 15,
 'substandard': 8,
 'fare': 169,
 'price': 4367,
 'review': 2833,
 'yelp': 1136,
 'observation': 12,
 'place': 23142,
 'visit': 2399,
 'hour': 4584,
 'minute': 3155,
 'dinner': 6079,
 'rush': 176,
 'rock': 971,
 'crab': 3401,
 'lb': 66,
 'california': 562,
 'salad': 3758,
 'choice': 1670,
 'soupchowder': 1,
 'look': 1206,
 'tray': 85,
 'wife': 2135,
 'saladchowder': 1,
 'combo': 339,
 'portion': 2394,
 'start': 814,
 'chowder': 3653,
 'side': 2957,
 'dish': 2597,
 'juicy': 507,
 'buttery': 252,
 'chunk': 285,
 'meat': 1928,
 'crack': 91,
 'shell': 456,
 'splinter': 2,
 'sign': 391,
 'perfection': 771,
 'amount': 1077,
 'work': 1373,
 'youre': 2353,
 'meal': 4381,
 'value': 367,
 'sbsc': 13,
 'trip': 1331,
 'proposition': 6,
 'spot': 4156,
 'wine': 9655,
 'merit': 7,
 'winery': 809,
 'label': 37,
 'premium': 42,
 'week': 1266,
 'location': 2984,
 'outdoor': 846,
 'patio': 2367,
 'staff': 7044,
 'super': 1006,
 '

In [176]:
list(dict(collections.Counter(all_noun).most_common(100)).keys())

['food',
 'place',
 'service',
 'time',
 'restaurant',
 'wine',
 'barbara',
 'bar',
 'staff',
 'order',
 'santa',
 'menu',
 'experience',
 'dinner',
 'night',
 'beer',
 'drink',
 'everything',
 'day',
 'hour',
 'meal',
 'price',
 'way',
 'friend',
 'star',
 'thing',
 'spot',
 'area',
 'burger',
 'flavor',
 'seafood',
 'server',
 'salad',
 'clam',
 'chowder',
 'cocktail',
 'lot',
 'crab',
 'table',
 'try',
 'minute',
 'bit',
 'taste',
 'view',
 'sauce',
 'wait',
 'get',
 'location',
 'side',
 'plate',
 'reservation',
 'chicken',
 'selection',
 'review',
 'lunch',
 'town',
 'eat',
 'room',
 'husband',
 'something',
 'dish',
 'waiter',
 'love',
 'fun',
 'year',
 'shrimp',
 'bread',
 'glass',
 'customer',
 'visit',
 'portion',
 'quality',
 'chip',
 'patio',
 'appetizer',
 'youre',
 'option',
 'family',
 'kind',
 'ambiance',
 'sb',
 'party',
 'taco',
 'dessert',
 'owner',
 'wife',
 'street',
 'nothing',
 'everyone',
 'fry',
 'oyster',
 'water',
 'bartender',
 'steak',
 'pasta',
 'lobster',


#### We still need to pick the words that are useful for the analysis

In [27]:
review_with_rating

,rating,review_words
0,5.0,"['bit', 'weary', 'trying', 'shellfish', 'compa..."
1,5.0,"['great', 'beautiful', 'spot', 'wine', 'tastin..."
2,3.0,"['tough', 'one', 'merit', 'wine', 'fairly', 'a..."
3,5.0,"['love', 'trying', 'fresh', 'seafood', 'pier',..."
4,5.0,"['super', 'delish', 'frill', 'great', 'sea', '..."
...,...,...
44688,5.0,"['favorite', 'brunch', 'spot', 'santa', 'barba..."
44689,5.0,"['combo', 'meat', 'plate', 'good', 'ton', 'foo..."
44690,2.0,"['table', 'got', 'many', 'weird', 'look', 'emp..."
44691,5.0,"['could', 'give', '5', 'star', 'would', 'reser..."


#### Merge the data having 1 business id for each row (combine reviews)

In [28]:
business = pd.read_csv("../../data/bars_list.csv").drop(columns = ["Unnamed: 0"])

In [29]:
business_CA = pd.DataFrame(business[business["state"] == "CA"])
business_CA = business_CA.sort_values(by = ["business_id"]).reset_index().drop(columns = ["index"])

In [30]:
review_i = []
bus_id = []

for row in range(len(bars_review)):
    if bars_review["business_id"][row] not in bus_id:
        review_i.append(row)
        bus_id.append(bars_review["business_id"][row])

In [31]:
len(review_i)

163

In [32]:
bars_review[44619:]

,business_id,stars_x,text
44619,zkMqKD6IZPcU10g0a2APmg,positive,Great atmosphere and Moses is very hospitable ...
44620,zkMqKD6IZPcU10g0a2APmg,positive,I love their selection of cigars. I've interac...
44621,zkMqKD6IZPcU10g0a2APmg,positive,Spent this past Saturday here for a birthday p...
44622,zkMqKD6IZPcU10g0a2APmg,positive,Best cigar lounge in America hands down!! Supe...
44623,zkMqKD6IZPcU10g0a2APmg,positive,This tobacco shop is a great jewel in the hear...
...,...,...,...
44688,zkMqKD6IZPcU10g0a2APmg,positive,First time to Santa Barbara and this shop but ...
44689,zkMqKD6IZPcU10g0a2APmg,positive,La Aroma De Havana is a great cigar lounge I w...
44690,zkMqKD6IZPcU10g0a2APmg,positive,Cigar and beer lover? Then it's a must stop wh...
44691,zkMqKD6IZPcU10g0a2APmg,positive,I would vacation in Santa Barbara just to smok...


In [33]:
combined_review = []
for i in range(len(review_i)):
    if i == len(review_i)-1:
        combined_review.append(list(bars_review["text"][review_i[i]:]))
    else:
        combined_review.append(list(bars_review["text"][review_i[i]:review_i[i+1]]))

In [34]:
combined_review[1]

["My husband and I stopped here once while we were in town last month. What really stood out to me was the comfortable, yet chic environment. They have a lot of very delicious sparkling wines- which you don't see a lot of in this area!\nWe will definitely be back!",
 "Girl friends and I made an unplanned stop at Riverbench and were blown away by the amazing time we had during our tasting, especially their sparkling brut rosé, I had to write a review for our pourer Jordie, who took very good care of us and was extremely passionate and knowledgeable about their products. She took the time to explain in detail each wine we chose for our flights. We also had 3 small dogs with us and she came out with dog treats to make sure they weren't forgotten. Also, the chocolate truffles were to die for. Needless to say we had an amazing time and left with a couple bottles of their fantastic wines and sparkling rosé. Definitely coming back the next time we're in Santa Barbara!",
 "I had a great time a

In [56]:
combined_review_df = pd.DataFrame()
for i in range(len(combined_review)):
    review = pd.DataFrame([combined_review[i]])
    review["merge"] = review[review.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    review = review["merge"].to_frame()
    combined_review_df = combined_review_df.append(review)
combined_review_df.columns = ["review_text"]
#combined_review_df = pd.concat([business_CA["business_id"], combined_review_df["review_text"]], axis = 1)
combined_review_df["business_id"] = list(business_CA["business_id"])
combined_review_df["stars"] = list(business_CA["stars"])
combined_review_df = combined_review_df.reset_index().drop(columns = ["index"])

In [58]:
combined_review_df

,review_text,business_id,stars
0,"Great beer selection, flat breads are great, g...",-BdYhP-12elmFV7oB1iv4A,5.0
1,My husband and I stopped here once while we we...,-e8RwknT5szoLk9uBZjzcw,4.0
2,My friends and I wanted to come to Llama dog b...,-ujBP1Dw0j1-Ffaz97-LXQ,4.0
3,Food here has always been fresh and delicious....,0CHIbqSkGWBr2KMkIUocEA,3.5
4,So happy we were able to get a reservation thi...,0ac5lGA_0wdKDnib3y59Xg,4.0
...,...,...,...
158,This was so awesome! The night market was a gr...,ySRkAFddd1tCUzRDjbECdg,4.0
159,I love coming here for their drinks and chicke...,ycbmvmBYk4w_ZmdrPL0tBQ,4.0
160,I came last night for the first time in awhile...,ywXa1pDqHPe9tOPx0yJFUg,3.0
161,"Loud,service at bar crazy. They rush around a...",zbrIMldF_O1ZQ0vpUaaa8A,4.0


In [59]:
combined_review_df.to_csv("combined_review.csv",index = False)

### 7. Look for the adjectives around the words (noun) that we chose to analyze

In [45]:
# Define a function for NLP

def txt_process(text):
    # 1. Deal with not ... case
    ## 1.1 some word include contractions (n't, 'll, 're)
    new_text = re.sub(r'won\'t', 'will not', text)
    new_text = re.sub(r'n\'t', ' not', new_text)
    new_text = re.sub(r'\'ll', ' will', new_text)
    new_text = re.sub(r'\'m', ' am', new_text)
    ## 1.2 take not ... case into account
    new_text = re.sub(r'not ', 'not_', new_text)
    
    # 2. Tokenize the sentence into words (also remove puncutation except "_" and lowercase)
    word_split = nltk.word_tokenize(new_text.lower().translate(str.maketrans('', '', string.punctuation.replace("_",""))))
    
    # 3. Remove stopwords
    stops = set(stopwords.words('english'))
    review_word = [word for word in word_split if word not in stops]
    
    """# 4. Stemming words
    sb_stemmer = SnowballStemmer("english",)
    review_stem_word = []
    for word in review_word:
        stemmer = sb_stemmer.stem(word)
        review_stem_word.append(stemmer)"""
    review_lemmatized_word = [lmtz.lemmatize(word) for word in review_word]
        
    return review_lemmatized_word

In [101]:
def adjWord(noun, processed_text):
    res = [[0]]*len(processed_text) # need list of string to do pos_tag
    for i in range(len(processed_text)):
        tag = nltk.pos_tag(processed_text[i])
        adjs = [tag[j-1][0] for j in range(len(processed_text[i])) if processed_text[i][j] == noun if tag[j-1][1] == "JJ" and j >= 1]
        # find the adjective around that noun
        res[i] = adjs
    return res

In [104]:
adjWord("beer", [["I", "love", "greatly", "good", "beer"]])

[['good']]

### 8. Analyze the sentiment score for the nouns.

In [110]:
combined_word_list = []
for i in range(len(combined_review_df)):
    combined_word_i = txt_process(combined_review_df["review_text"][i])
    combined_word_list.append(combined_word_i)

In [109]:
wine_adjective = adjWord("wine", combined_word_list)
wine_adjective

[['avoid'],
 ['fantastic',
  'valuegreat',
  'good',
  'enjoyed',
  'f',
  'counter',
  'atmosphere',
  'good',
  'couple',
  'returnthe',
  'delicious',
  'favorite',
  'whole',
  'knowledgeable',
  'different',
  'general',
  'miss',
  'bottle',
  'single',
  'sparkling',
  'crowded',
  'knowledgeable',
  'wine',
  'loved',
  'shot',
  'cozy',
  'friendly',
  'start',
  'great',
  'great',
  'helpful',
  'star',
  'good',
  'backfantastic',
  'exceptional',
  'exceptional',
  'favorite',
  'good',
  'white',
  'beautiful',
  'drinkable',
  'delicious',
  'steep',
  'incredible',
  'bottle',
  'great',
  'affordable',
  'versed',
  'great',
  'amazing',
  'stylish',
  'white',
  'non',
  'amazing',
  'next',
  'sparkling',
  'white',
  'spite',
  'high',
  'many',
  'distancegreat',
  'knowledgeable',
  'spark',
  'single',
  'sparking',
  'red',
  'red',
  'red',
  'liked',
  'decentriverbenchs',
  'excellent',
  'pour',
  'great',
  'many',
  'canyon',
  'nice',
  'white',
  'pour',

In [115]:
for word in dict(Counter(wine_adjective[1])):
    print(word)

fantastic
valuegreat
good
enjoyed
f
counter
atmosphere
couple
returnthe
delicious
favorite
whole
knowledgeable
different
general
miss
bottle
single
sparkling
crowded
wine
loved
shot
cozy
friendly
start
great
helpful
star
backfantastic
exceptional
white
beautiful
drinkable
steep
incredible
affordable
versed
amazing
stylish
non
next
spite
high
many
distancegreat
spark
sparking
red
liked
decentriverbenchs
excellent
pour
canyon
nice
placegreat
whit
tasting
worth
drank
informative
urban
bad
particular
le
insultingwonderful


In [117]:
adj_classifier.keys()

dict_keys(['weary', 'shellfish', 'many', 'monterey', 'feast', 'captive', 'high', 'perennial', 'good', 'initial', 'small', 'plan', 'peak', 'special', 'local', 'awesome', 'great', 'hearty', 'much', 'shrimp', 'bisque', 'prattling', 'glorious', 'white', 'thick', 'separate', 'satisfied', 'overall', 'favorite', 'safe', 'beautiful', 'wine', 'tried', 'tough', 'average', 'understand', 'lable', 'available', 'wonderful', 'nicer', 'enjoy', 'super', 'expensive', 'free', 'winery', 'give', 'complimentary', 'carr', 'new', 'limited', 'avoid', 'urban', 'fresh', 'wharf', 'gorgeous', 'top', 'wrong', 'lunchearly', 'table', 'outdoor', 'right', 'sunscreen', 'salad', 'steal', 'huge', 'delicious', 'simple', 'crab', 'cup', 'fried', 'cole', 'greasy', 'sun', 'u', 'extra', 'tea', 'hot', 'make', 'sure', 'last', 'minute', 'upset', 'large', 'pier', 'late', 'sour', 'pear', 'different', 'excellent', 'flavor', 'nonexistent', 'outside', 'ocean', 'prepared', 'seated', 'worth', 'selective', 'hit', 'fish', 'seafood', 'littl

In [121]:
wine_adjective[1]

['fantastic',
 'valuegreat',
 'good',
 'enjoyed',
 'f',
 'counter',
 'atmosphere',
 'good',
 'couple',
 'returnthe',
 'delicious',
 'favorite',
 'whole',
 'knowledgeable',
 'different',
 'general',
 'miss',
 'bottle',
 'single',
 'sparkling',
 'crowded',
 'knowledgeable',
 'wine',
 'loved',
 'shot',
 'cozy',
 'friendly',
 'start',
 'great',
 'great',
 'helpful',
 'star',
 'good',
 'backfantastic',
 'exceptional',
 'exceptional',
 'favorite',
 'good',
 'white',
 'beautiful',
 'drinkable',
 'delicious',
 'steep',
 'incredible',
 'bottle',
 'great',
 'affordable',
 'versed',
 'great',
 'amazing',
 'stylish',
 'white',
 'non',
 'amazing',
 'next',
 'sparkling',
 'white',
 'spite',
 'high',
 'many',
 'distancegreat',
 'knowledgeable',
 'spark',
 'single',
 'sparking',
 'red',
 'red',
 'red',
 'liked',
 'decentriverbenchs',
 'excellent',
 'pour',
 'great',
 'many',
 'canyon',
 'nice',
 'white',
 'pour',
 'placegreat',
 'whit',
 'white',
 'enjoyed',
 'tasting',
 'worth',
 'drank',
 'informati

In [199]:
def key_score(keyword):
    keyword_adj = adjWord(keyword, combined_word_list)
    
    count_positive = []
    count_negative = []
    
    for adj_list in keyword_adj:
        sum_pos = 0
        sum_neg = 0
        for adj in adj_list:
            # First, check if the adjective can be sentimental analyzed
            if adj in adj_classifier.keys():
                # define positivity / negativity of the word
                if adj_classifier[adj] == 'positive':
                    sum_pos += 1
                if adj_classifier[adj] == 'negative':
                    sum_neg += 1
        count_positive.append(sum_pos)
        count_negative.append(sum_neg)
        
    return count_positive, count_negative

In [200]:
def keyword_sentiment_df(keyword, outfile):
    df = pd.DataFrame(business_CA["business_id"])
    # add columns
    df["stars"] = list(business_CA["stars"])
    df["positive_count"] = key_score(keyword)[0]
    df["negative_count"] = key_score(keyword)[1]
    # calculate the percentage sentiment score
    score = 100*np.array(count_positive)/(np.array(count_positive)+np.array(count_negative))
    df["sentiment_score_%"] = score
    df.to_csv(outfile + ".csv")
    return df

In [203]:
keyword_sentiment_df("wine", "wine_sentiment")

C:\Users\boom-\AppData\Local\Temp/ipykernel_64356/3422114300.py:8: RuntimeWarning: invalid value encountered in true_divide
  score = 100*np.array(count_positive)/(np.array(count_positive)+np.array(count_negative))


,business_id,stars,positive_count,negative_count,sentiment_score_%
0,-BdYhP-12elmFV7oB1iv4A,5.0,1,0,100.000000
1,-e8RwknT5szoLk9uBZjzcw,4.0,86,2,97.727273
2,-ujBP1Dw0j1-Ffaz97-LXQ,4.0,7,0,100.000000
3,0CHIbqSkGWBr2KMkIUocEA,3.5,12,0,100.000000
4,0ac5lGA_0wdKDnib3y59Xg,4.0,3,0,100.000000
...,...,...,...,...,...
158,ySRkAFddd1tCUzRDjbECdg,4.0,4,0,100.000000
159,ycbmvmBYk4w_ZmdrPL0tBQ,4.0,2,0,100.000000
160,ywXa1pDqHPe9tOPx0yJFUg,3.0,0,0,NaN
161,zbrIMldF_O1ZQ0vpUaaa8A,4.0,25,2,92.592593


In [202]:
df

,business_id,stars,positive_count,negative_count
0,-BdYhP-12elmFV7oB1iv4A,5.0,1,0
1,-e8RwknT5szoLk9uBZjzcw,4.0,86,2
2,-ujBP1Dw0j1-Ffaz97-LXQ,4.0,7,0
3,0CHIbqSkGWBr2KMkIUocEA,3.5,12,0
4,0ac5lGA_0wdKDnib3y59Xg,4.0,3,0
...,...,...,...,...
158,ySRkAFddd1tCUzRDjbECdg,4.0,4,0
159,ycbmvmBYk4w_ZmdrPL0tBQ,4.0,2,0
160,ywXa1pDqHPe9tOPx0yJFUg,3.0,0,0
161,zbrIMldF_O1ZQ0vpUaaa8A,4.0,25,2


#### Create dataframes for each keyword sentiments

In [204]:
keyword_list = ['menu','service','price','place','burger','seafood'
                ,'salad','clam','chowder','crab','chicken','bread'
                ,'chip','appetizer','taco','dessert','oyster','steak'
                ,'pasta','lobster','wine','beer','cocktail','water'
                ,'staff','table','reservation','waiter','bartender'
                ,'waitress']

for i in keyword_list:
    outfile = i+"_sentiment"
    keyword_sentiment_df(i, outfile)

C:\Users\boom-\AppData\Local\Temp/ipykernel_64356/3422114300.py:8: RuntimeWarning: invalid value encountered in true_divide
  score = 100*np.array(count_positive)/(np.array(count_positive)+np.array(count_negative))
